In [1]:
import pandas as pd
import numpy as np

In [2]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])

In [3]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five  

1. 

In [6]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_2010']
right = df2011.groupby(df2011.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
right.columns = ['symb', 'max_2011']
pd.merge(left, right, on = ['symb'], how = 'inner')

,symb,max_2010,max_2011
0,A,29.9857,37.6109
1,AA,52.3500,54.3900
2,AAME,2.0500,2.2200
3,AAN,23.8000,29.1600
4,AAON,8.6697,10.5955
...,...,...,...
3330,ZIXI,4.4900,4.7500
3331,ZN,7.2000,6.9800
3332,ZNH,38.6500,35.4000
3333,ZTR,15.8400,14.5600


2. 

In [12]:
right = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
right.columns = ['symb', 'last2010']
left = df2011.groupby(df2011.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
left.columns = ['symb', 'last2011']
pd.merge(left, right, on = ['symb'], how = 'left')

,symb,last2011,last2010
0,A,2011-12-30,2010-12-31
1,AA,2011-12-30,2010-12-31
2,AAME,2011-12-30,2010-12-31
3,AAN,2011-12-30,2010-12-31
4,AAON,2011-12-30,2010-12-31
...,...,...,...
3529,ZNGA,2011-12-30,NaT
3530,ZNH,2011-12-30,2010-12-31
3531,ZTR,2011-12-30,2010-12-31
3532,ZUMZ,2011-12-30,2010-12-31


3.

In [15]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = df2010.loc[:, ['retdate']].drop_duplicates()
pd.merge(left, right, how = 'cross')

,symb,retdate
0,AAME,2010-04-07
1,AAME,2010-04-13
2,AAME,2010-06-11
3,AAME,2010-11-23
4,AAME,2010-02-01
...,...,...
847219,EDF,2010-11-12
847220,EDF,2010-02-24
847221,EDF,2010-08-13
847222,EDF,2010-06-08


4. 

In [20]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = df2010.loc[:, ['retdate']].drop_duplicates()
pd.merge(left, right, how = 'cross').shape[0]

847224

In [19]:
df2010.shape[0]

816066

5. 

In [26]:
left = df2010.groupby(df2010.loc[:, 'retdate'].dt.strftime('%B')).agg({'retdate': ['count']}).reset_index()
left.columns = ['Month', 'nrows2010']
right = df2011.groupby(df2011.loc[:, 'retdate'].dt.strftime('%B')).agg({'retdate': ['count']}).reset_index()
right.columns = ['Month', 'nrows2011']
pd.merge(left, right, on = ['Month'], how = 'inner')

,Month,nrows2010,nrows2011
0,April,67243,67995
1,August,71731,79524
2,December,73041,73458
3,February,60116,63859
4,January,60003,66714
5,July,68232,68855
6,June,71170,75446
7,March,73087,77618
8,May,64394,71783
9,November,69398,73211


# Main Problems

1. 

In [6]:
left = df2010.loc[:, ['symb']].drop_duplicates()
left.columns = ['symb_2010']
right = df2011.loc[:, ['symb']].drop_duplicates()
right.columns = ['symb_2011']
temp = pd.merge(left, right, left_on = 'symb_2010', right_on = 'symb_2011', how = 'left', indicator = True)
temp.loc[(temp.loc[:, '_merge'] == 'left_only'), 'symb_2010'].count()

27

2.

In [9]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['min']})
left.columns = ['symb', 'firstday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
pd.merge(left, right, left_on = ['symb', 'firstday'], right_on = ['symb', 'retdate'], how = 'left')

,symb,firstday,retdate,cls
0,A,2010-01-04,2010-01-04,22.3891
1,AA,2010-01-04,2010-01-04,49.9500
2,AAME,2010-01-04,2010-01-04,1.3200
3,AAN,2010-01-04,2010-01-04,18.5400
4,AAON,2010-01-04,2010-01-04,5.9230
...,...,...,...,...
3357,ZIXI,2010-01-04,2010-01-04,1.7800
3358,ZN,2010-01-04,2010-01-04,7.2000
3359,ZNH,2010-01-04,2010-01-04,15.9800
3360,ZTR,2010-01-04,2010-01-04,15.6800


3. 

In [13]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['min']})
left.columns = ['symb', 'firstday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
new_left = pd.merge(left, right, left_on = ['symb', 'firstday'], right_on = ['symb', 'retdate'], how = 'left').loc[:, ['symb', 'cls']].rename(columns = {'cls': 'firstdaycls'})
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
left.columns = ['symb', 'lastday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
new_right = pd.merge(left, right, left_on = ['symb', 'lastday'], right_on = ['symb', 'retdate'], how = 'left').loc[:, ['symb', 'cls']].rename(columns = {'cls': 'lastdaycls'})
pd.merge(new_left, new_right, how = 'inner', on = 'symb')

,symb,firstdaycls,lastdaycls
0,A,22.3891,29.6352
1,AA,49.9500,46.1700
2,AAME,1.3200,2.0300
3,AAN,18.5400,20.3900
4,AAON,5.9230,8.3585
...,...,...,...
3357,ZIXI,1.7800,4.2700
3358,ZN,7.2000,4.7800
3359,ZNH,15.9800,30.5100
3360,ZTR,15.6800,14.2400


4. 

In [10]:
left = df2010.loc[(df2010.loc[:, 'retdate'].dt.strftime('%A') == 'Monday'), ['symb', 'retdate', 'cls']]
right = df2010.loc[(df2010.loc[:, 'retdate'].dt.strftime('%A') == 'Tuesday'), ['symb', 'retdate', 'cls']].rename(columns={'cls': 'cls_nextday'})
right.loc[:, 'daybefore'] = right.loc[:, 'retdate'] - pd.Timedelta(days = 1)
right
pd.merge(left, right.drop(columns = ['retdate']), left_on = ['symb', 'retdate'], right_on = ['symb', 'daybefore'], how = 'inner').drop(columns = ['daybefore'])

,symb,retdate,cls,cls_nextday
0,AAME,2010-02-01,1.3500,1.3900
1,AAON,2010-02-01,6.0889,6.2015
2,AAPL,2010-02-01,27.8186,27.9800
3,AAWW,2010-02-01,37.3700,38.3400
4,AAXJ,2010-02-01,53.0700,53.4200
...,...,...,...,...
152242,YUM,2010-01-25,34.2800,34.3300
152243,ZB-A,2010-01-25,16.2500,16.3500
152244,ZF,2010-01-25,12.8800,12.8800
152245,ZNH,2010-01-25,16.9400,16.4600


5. 

In [33]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = dffnd.loc[:, ['tic']].drop_duplicates()
temp = pd.merge(left, right, left_on='symb', right_on = 'tic', how = 'outer', indicator = True)
nboth = temp.loc[(temp.loc[:, '_merge'] == 'both'), '_merge'].count()
n2010 = temp.loc[(temp.loc[:, '_merge'] == 'left_only'), '_merge'].count()
nfnd = temp.loc[(temp.loc[:, '_merge'] == 'right_only'), '_merge'].count()
pd.DataFrame({'both': nboth, '2010': n2010, 'fnd': nfnd}, index = [1])

,both,2010,fnd
1,3119,243,10060


6. 

In [28]:
temp = pd.merge(left = df2010.loc[:, ['symb']].drop_duplicates(), right = dffnd.loc[:, ['tic']].drop_duplicates(), how = 'outer', left_on = ['symb'], right_on = ['tic'], indicator= True)
temp.groupby(temp.loc[:, '_merge']).agg({'_merge': ['count']}).reset_index()

_merge       
               count
0   left_only    243
1  right_only  10060
2        both   3119

7. 

In [39]:
left = df2010.groupby(df2010.loc[:, 'retdate'].dt.strftime('%A')).agg({'cls': ['mean']}).reset_index()
left.columns = ['dow', 'cls_2010']
right = df2011.groupby(df2011.loc[:, 'retdate'].dt.strftime('%A')).agg({'cls': ['mean']}).reset_index()
right.columns = ['dow', 'cls_2011']
pd.merge(left, right, on = 'dow', how = 'inner')

,dow,cls_2010,cls_2011
0,Friday,2684.083145,1876.194780
1,Monday,2732.947968,1885.520858
2,Thursday,2698.283494,1908.122748
3,Tuesday,2704.411781,1895.687620
4,Wednesday,2699.352646,1893.319519


In [54]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_cls']
left.loc[(left.loc[:, 'max_cls'] < 50), 'type'] = 'below50'
left.loc[(left.loc[:, 'max_cls'] > 50) & (left.loc[:, 'max_cls'] < 100), 'type'] = 'between'
left.loc[(left.loc[:, 'max_cls'] > 100), 'type'] = 'above100'
right = dffnd.loc[(dffnd.loc[:, 'fyear'] == 2010), :].groupby(dffnd.loc[:, 'tic']).agg({'netinc': ['max']}).reset_index()
right.columns = ['symb', 'netinc']
right

,symb,netinc
0,0015B,30.998
1,0030B,22.828
2,0032A,-3530.000
3,0033A,-2812.000
4,0038A,204.982
...,...,...
11068,ZUT.,NaN
11069,ZX,49.662
11070,ZYCI,NaN
11071,ZYXI,0.350


8. 

In [57]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_cls']
left.loc[(left.loc[:, 'max_cls'] < 50), 'type'] = 'below50'
left.loc[(left.loc[:, 'max_cls'] > 50) & (left.loc[:, 'max_cls'] < 100), 'type'] = 'between'
left.loc[(left.loc[:, 'max_cls'] > 100), 'type'] = 'above100'
right = dffnd.loc[(dffnd.loc[:, 'fyear'] == 2010), :].groupby(dffnd.loc[:, 'tic']).agg({'netinc': ['max']}).reset_index()
right.columns = ['symb', 'netinc']
right = right.loc[~(right.loc[:, 'netinc'].isna()), :]
out = pd.merge(left, right, on = 'symb', how = 'inner')
out.groupby(out.loc[:, 'type'], as_index = False).agg({'netinc': ['mean']})

,type,netinc
,,mean
0,above100,963.670744
1,below50,325.222616
2,between,1482.581835
